# Data Cleaning 

#### 1. Import pandas library.

In [1]:
import pandas as pd

#### 2. Import the users table.

In [2]:
users = pd.read_csv("../data/users.csv")

In [3]:
users.head()

,Unnamed: 0,userId,Reputation,Views,UpVotes,DownVotes
0,0,-1,1,0,5007,1920
1,1,2,101,25,3,0
2,2,3,101,22,19,0
3,3,4,101,11,0,0
4,4,5,6792,1145,662,5


#### 3. Rename Id column to userId.

In [ ]:
#no need. The column in the csv file is "userID"

#### 4. Import the posts table. 

In [4]:
posts = pd.read_csv("../data/posts.csv")

In [5]:
posts.head()

,Unnamed: 0,PostId,userId,Score,ViewCount,CommentCount
0,0,1,8.0,23,1278.0,1
1,1,2,24.0,22,8198.0,1
2,2,3,18.0,54,3613.0,4
3,3,4,23.0,13,5224.0,2
4,4,5,23.0,81,NaN,3


#### 5. Rename Id column to postId and OwnerUserId to userId.

#### 6. Define new dataframes for users and posts with the following selected columns:
**users_sliced columns**: userId, Reputation, Views, UpVotes, DownVotes  
**posts_sliced columns**: postId, Score, userId, ViewCount, CommentCount

In [12]:
users_sliced = users[["userId", "Reputation", "Views", "UpVotes", "DownVotes"]]


In [22]:
posts_sliced = posts[["PostId", "Score", "userId", "ViewCount", "CommentCount"]]

#### 7. Merge the two dataframes created in the step above (8), users_sliced and posts_sliced. 
You will need to make a [merge](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.merge.html) of posts and users dataframes.

In [43]:
users_with_posts = users_sliced.merge(posts_sliced, how="inner", right_on="userId", left_on="userId")

#### 8. How many missing values do you have in your merged dataframe? On which columns?

In [44]:
users_with_posts.isnull().sum()

userId              0
Reputation          0
Views               0
UpVotes             0
DownVotes           0
PostId              0
Score               0
ViewCount       48396
CommentCount        0
dtype: int64

#### 9. You will need to make something with missing values.  Will you clean or filling them? Explain. 
**Remember** to check the results of your code before going to the next step.

In [45]:
# I think it is better to fill them with 0s. 
users_with_posts["ViewCount"].fillna(0, inplace=True)

#### 10. Adjust the data types in order to avoid future issues. Which ones should be changed? 

In [47]:
users_with_posts.dtypes
# the ViewCount dtype is float64. It should be changed to int64. 
# We should not allow view counts in decimals.

users_with_posts = users_with_posts.astype("int64")